# How to compute TESS asteroid ephemeris using Spice?

This notebooks contains a snippet of code demonstrating how one can use the Spice Toolkit to compute the position of asteroids as seen from the vantage point of TESS.

## Step 1: Loading orbital elements

For the purpose of this example, we define a minimalist `Target` class to hold the Sun-centered orbital elements of a target.  We also add a `Target.from_horizons()` method to load the elements for a target from JPL Horizons.

In [1]:
import attr
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy import units as u
from astropy.units import Quantity
from astropy.coordinates import SkyCoord
import spiceypy as spice

# Standard gravitational parameter of the Sun (source: `gm_de431.tpc`)
GM_SUN = Quantity(1.3271244004193938E+11, unit="km^3/s^2")

@attr.s
class Target():
    """Data container holding the orbital elements of a minor planet target."""
    # Target name
    name : str = attr.ib()
    # Eccentricity
    e : float = attr.ib()
    # Semimajor axis
    a : float = attr.ib()
    # Inclination
    incl : float = attr.ib()
    # Longitude of the ascending node
    node : float = attr.ib()
    # Argument of periapsis
    peri : float = attr.ib()
    # Mean anomaly at epoch
    m : float = attr.ib()
    # Epoch i.e. reference time
    epoch : float = attr.ib()

    @property
    def q(self):
        """Periapsis distance, i.e. (1-e)*a"""
        return (1-self.e.value)*self.a
      
    @classmethod
    def from_horizons(cls, id, epoch=None, location="@sun"):
        """Load orbital elements from JPL Horizons.
        
        Parameters
        ----------
        id : str
            Name, number, or designation of the object.
        epoch : Time
            Reference time for the osculating orbital elements.
            If no epochs is provided, the current time is used.
        location : str
            JPL Horizons location code for the center body of the
            orbital elements. Defaults to the Sun ("500@10").
        """
        obj = Horizons(id=id, location=location, epochs=epoch)
        el = obj.elements()
        result = cls(name=id,
                     e=u.Quantity(el['e'].quantity.value[0], unit=''),
                     a=el['a'].quantity[0],
                     incl=el['incl'].quantity[0],
                     node=el['Omega'].quantity[0],
                     peri=el['w'].quantity[0],
                     m=el['M'].quantity[0],
                     epoch=el['datetime_jd'].quantity[0])
        result._elements = el
        return result

    def to_spice(self):
        """Returns the orbital elements in the array format required by SPICE."""
        # We could also use
        # `epoch_et = spice.str2et(f"JDTDB {self.epoch.value}")`
        # but we prefer using AstroPy tdb=>utc conversion to avoid
        # having to load the spice leap second kernel in this class.
        epoch_et = spice.str2et(Time(self.epoch.value, format='jd', scale='tdb').utc.iso)
        elements = [self.q.to("km").value,
                    self.e.value,
                    self.incl.to("rad").value,
                    self.node.to("rad").value,
                    self.peri.to("rad").value,
                    self.m.to("rad").value,
                    epoch_et,
                    GM_SUN.value]
        return elements

**Example use:**

In [2]:
epoch = Time(2458848.0, format='jd', scale='utc')
targ = Target.from_horizons("Ceres", epoch=epoch)
targ

Target(name='Ceres', e=<Quantity 0.07686737>, a=<Quantity 2.76930498 AU>, incl=<Quantity 10.59130887 deg>, node=<Quantity 80.3012895 deg>, peri=<Quantity 73.81009657 deg>, m=<Quantity 129.99442539 deg>, epoch=<Quantity 2458848. d>)

## Step 2: Converting orbital elements to (ra, dec) ephemeris using Spice

In [3]:
def get_viewing_vector(elements, et, location, correct_lt=True, correct_stelab=False):
    """Returns the ecliptic XYZ viewing vector pointing from the observer to the target."""
    # Load planetary positions
    kernels = ["de432s.bsp", "TESS_EPH_PRE_2YEAR_2018171_01.bsp"]
    spice.furnsh(kernels)

    # Compute state vector Sun=>target
    target_state = spice.conics(elements, et)
    
    # Compute state vector Sun=>observer
    observer_state, _ = spice.spkezr(location, et, "ECLIPJ2000", "NONE", "SUN")
    
    # Apply one-way light time correction
    if correct_lt:
        # Compute distance observer=>target
        dist = spice.vdist(observer_state[:3], target_state[:3])
        light_time_correction = dist / spice.clight()
        target_state = spice.conics(elements, et - light_time_correction)

    # Compute vector observer=>target
    observer2target = spice.vsubg(target_state, observer_state, 6)
        
    # Correct the vector for stellar aberration
    if correct_stelab:
        ssb2observer, _ = spice.spkezr(location, et, "ECLIPJ2000", "NONE", "SOLAR SYSTEM BARYCENTER")
        observer2target[:3] = spice.stelab(observer2target[:3], ssb2observer[3:])
   
    return observer2target

    
def get_ephemeris(target, utc, location="399", correct_lt=True, correct_stelab=False):
    """Returns the equatorial ephemeris of a target at a requested time `utc`."""
    # Allow using "TESS" as location string instead of "-95"
    spice.boddef("TESS", -95)

    # Convert the time stamp to the Spice format (ephemeris seconds past J2000)
    kernels = ["naif0012.tls"]
    spice.furnsh(kernels)
    et = spice.str2et(utc)

    # Obtain the orbital elements of the target in the Spice format
    elements = target.to_spice()

    # Compute the XYZ vector from observer to target
    vector = get_viewing_vector(elements,
                                et,
                                location=location,
                                correct_lt=correct_lt,
                                correct_stelab=correct_stelab)
    
    # Transform the viewing vector from ecliptic XYZ coordinates
    # into spherical Earth equatorial coordinates
    xform = spice.sxform("ECLIPJ2000", "J2000", et)
    vector_icrf = spice.mxvg(xform, vector, 6, 6)
    dist, ra_radians, dec_radians = spice.recrad(vector_icrf[:3])

    return SkyCoord(ra_radians*u.rad, dec_radians*u.rad)

**Example use:**

In [4]:
crd_spice = get_ephemeris(targ, utc=epoch.iso, location="TESS")
crd_spice

<SkyCoord (ICRS): (ra, dec) in deg
    (289.01780341, -26.28014981)>

## Step 3a: Verification for coeval orbital elements

Let's compare our prediction against the ephemeris produced by JPL Horizons:

In [5]:
# Load ephemeris directly from Horizons
jpl = Horizons(id="Ceres", epochs=epoch.jd, location="@TESS").ephemerides(quantities=1)
crd_horizons = SkyCoord(jpl['RA'], jpl['DEC'])
crd_horizons

<SkyCoord (ICRS): (ra, dec) in deg
    [(289.01782, -26.28014)]>

In [6]:
# Compare the offset between our ephemeris and that produced by Horizons
crd_spice.separation(crd_horizons)

<Angle [1.78230997e-05] deg>

Success! Our result match within 0.1 arcsecond!

## Step 3b: Verification for non-coeval orbital elements

Now, let's repeat this verification using orbital elements that were requested for a different reference epoch:

In [7]:
targ = Target.from_horizons("Ceres", epoch=epoch - 365)
targ

Target(name='Ceres', e=<Quantity 0.07576353>, a=<Quantity 2.76798982 AU>, incl=<Quantity 10.59380341 deg>, node=<Quantity 80.30651591 deg>, peri=<Quantity 73.33800644 deg>, m=<Quantity 52.50419682 deg>, epoch=<Quantity 2458483. d>)

In [8]:
crd_spice = get_ephemeris(targ, utc=epoch.iso, location="TESS")
crd_spice

<SkyCoord (ICRS): (ra, dec) in deg
    (289.02676926, -26.28030433)>

In [9]:
crd_spice.separation(crd_horizons)

<Angle [0.00802594] deg>

Oops, the ephemeris is now inconsistent by 28 arcseconds.  This is because Spice only applied 2-body propagation to infer the position of the asteroid from the non-coeval orbital elements.  In contrast, Horizons applied N-body propagation.